**Étape 1: Importation des bibliothèques**

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import LearningRateScheduler, Callback
import tensorflow_datasets as tfds


**Étape 2: Chargement et préparation du dataset Malaria**

On utilise le dataset Malaria de TensorFlow Datasets. Il contient des images de cellules infectées et non infectées, que l’on prétraite (redimensionnement et normalisation).

In [6]:
# Charger le dataset Malaria
(ds_train, ds_test), ds_info = tfds.load('malaria',
                                         split=['train[:80%]', 'train[80%:]'],
                                         shuffle_files=True,
                                         as_supervised=True,
                                         with_info=True)

# Redimensionner les images et les normaliser
IMG_SIZE = 224
BATCH_SIZE = 32

def preprocess_image(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = image / 255.0  # Normalisation
    return image, label

# Appliquer le prétraitement
ds_train = ds_train.map(preprocess_image).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)
ds_test = ds_test.map(preprocess_image).batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)


Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:   0%|          | 0/1 [00:00<?, ? url/s]
Extraction completed...: 0 file [00:00, ? file/s]
Dl Size...: 0 MiB [00:00, ? MiB/s]
Dl Completed...:   0%|          | 0/1 [00:00<?, ? url/s]


DownloadError: Failed to get url https://data.lhncbc.nlm.nih.gov/public/Malaria/cell_images.zip. HTTP code: 403.

**Étape 3: Création du modèle EfficientNet-B0**

EfficientNet-B0 est utilisé avec des poids pré-entraînés d’ImageNet. On ajoute des couches pour la classification binaire (infecté/non infecté).

In [ ]:
def create_model():
    base_model = EfficientNetB0(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3), weights='imagenet')
    base_model.trainable = True  # Fine-tuning

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Classification binaire
    ])

    return model


**Étape 4: Scheduler pour Cosine Annealing**

On utilise le Cosine Annealing pour moduler dynamiquement le taux d'apprentissage.

In [ ]:
def cosine_annealing(epoch):
    initial_lr = 0.001
    epochs = 50
    lr_min = 1e-6
    return lr_min + (initial_lr - lr_min) * (1 + np.cos(np.pi * epoch / epochs)) / 2


**Étape 5: Callback pour enregistrer les Snapshots**

Le modèle est sauvegardé à la fin des époques 10, 20, 30, 40 et 50.

In [ ]:
class SnapshotCallback(Callback):
    def __init__(self, snapshots):
        super(SnapshotCallback, self).__init__()
        self.snapshots = snapshots

    def on_epoch_end(self, epoch, logs=None):
        if epoch in [9, 19, 29, 39, 49]:  # Snapshots aux 10e, 20e, 30e, 40e, 50e époques
            snapshot = self.model.get_weights()
            self.snapshots.append(snapshot)

snapshots = []
cosine_annealing_schedule = LearningRateScheduler(cosine_annealing)
snapshot_callback = SnapshotCallback(snapshots)


**Étape 6: Compilation et entraînement du modèle**

Le modèle est compilé et entraîné avec la stratégie Snapshot Ensemble.

In [ ]:
model = create_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(ds_train,
                    validation_data=ds_test,
                    epochs=50,
                    callbacks=[cosine_annealing_schedule, snapshot_callback])


**Étape 7: Création d'ensembles avec les Snapshots**

On combine les snapshots pour créer différents ensembles de modèles et les évaluer.

In [ ]:
def ensemble_predictions(models, input_data):
    predictions = [model.predict(input_data) for model in models]
    return tf.reduce_mean(predictions, axis=0)

# Créer des modèles pour chaque snapshot
snapshot_models = [create_model() for _ in snapshots]

# Charger les poids des snapshots
for i, snapshot in enumerate(snapshots):
    snapshot_models[i].set_weights(snapshot)

# Combinaisons des ensembles de snapshots
ensemble_pred_1 = ensemble_predictions(snapshot_models[:5], ds_test)  # Snapshots [1, 2, 3, 4, 5]
ensemble_pred_2 = ensemble_predictions(snapshot_models[1:], ds_test)  # Snapshots [2, 3, 4, 5]
ensemble_pred_3 = ensemble_predictions(snapshot_models[2:], ds_test)  # Snapshots [3, 4, 5]
ensemble_pred_4 = ensemble_predictions(snapshot_models[2:4], ds_test)  # Snapshots [3, 4]
ensemble_pred_5 = ensemble_predictions(snapshot_models[3:5], ds_test)  # Snapshots [4, 5]


**Étape 8: Évaluation des ensembles**

On évalue les performances de chaque ensemble en termes d'exactitude.

In [ ]:
def evaluate_ensemble(ensemble_predictions, ds_test):
    y_true = []
    y_pred = []
    for images, labels in ds_test:
        y_true.extend(labels.numpy())
        y_pred.extend(tf.round(ensemble_predictions(images)).numpy())

    accuracy = np.mean(np.array(y_true) == np.array(y_pred))
    return accuracy

# Calculer l'exactitude pour chaque ensemble
accuracy_ensemble_1 = evaluate_ensemble(ensemble_pred_1, ds_test)
accuracy_ensemble_2 = evaluate_ensemble(ensemble_pred_2, ds_test)
accuracy_ensemble_3 = evaluate_ensemble(ensemble_pred_3, ds_test)
accuracy_ensemble_4 = evaluate_ensemble(ensemble_pred_4, ds_test)
accuracy_ensemble_5 = evaluate_ensemble(ensemble_pred_5, ds_test)

print(f"Exactitude Ensemble 1 (1,2,3,4,5): {accuracy_ensemble_1}")
print(f"Exactitude Ensemble 2 (2,3,4,5): {accuracy_ensemble_2}")
print(f"Exactitude Ensemble 3 (3,4,5): {accuracy_ensemble_3}")
print(f"Exactitude Ensemble 4 (3,4): {accuracy_ensemble_4}")
print(f"Exactitude Ensemble 5 (4,5): {accuracy_ensemble_5}")


**Étape 9: Visualisation avec GradCAM**

Utilisez GradCAM pour visualiser les zones activées par le modèle lors de la prédiction.

In [ ]:
import cv2

def get_gradcam_heatmap(model, image, last_conv_layer_name):
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])

    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(image)
        loss = predictions[:, 0]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    heatmap = tf.squeeze(heatmap).numpy()

    return heatmap

# Appliquer GradCAM sur une image de test
last_conv_layer_name = 'top_conv'
image = next(iter(ds_test.take(1)))[0]

heatmap = get_gradcam_heatmap(model, image, last_conv_layer_name)

# Superposer la heatmap sur l'image originale
image = image.numpy().squeeze()
heatmap = cv2.resize(heatmap, (image.shape[1], image.shape[0]))
heatmap = np.uint8(255 * heatmap)

plt.imshow(image)
plt.imshow(heatmap, cmap='jet', alpha=0.4)
plt.show()


**Étape 10: Sauvegarder chaque modèle Snapshot**

Après avoir créé les différents modèles de snapshots et évalué leurs performances, vous pouvez les sauvegarder pour une future utilisation.

In [ ]:
# Sauvegarder chaque snapshot au format .h5
for i, snapshot_model in enumerate(snapshot_models):
    h5_target_path = f'datas/snapshot_model_{i+1}.h5'
    snapshot_model.save(h5_target_path)
    print(f'Modèle snapshot {i+1} sauvegardé au format .h5 dans {h5_target_path}')